# Pandas II: Manipulating data frames

Last time we learned about the ability of Python to store and manipulate tabular data with the Pandas package.

Tabular data is quite common in genomic analyses.  You can imagine wanting to create and manipulate a table like the following:

| Gene | expression in condition 1 | expression in condition 2 | log2 fold change | pvalue |
|------|---------------------------|---------------------------|------------------|--------|
|GeneA |       100                 | 200                       |        1         |   6e-5 |
|GeneB |    50                     | 12.5                      |        -2        |  2e-6  |
|GeneC |       40                  |   45                      |   0.17           | 0.55   |


Ending up with a table like this often involves **merging** smaller tables together, **selecting** different columns of data from a data frame, **filtering** for rows that match specific conditions, manipulating the **shape** of the table between long and wide formats, and finally, plotting outputs.

In this lesson, we will learn how to perform each of these tasks.  

## Selecting rows based on conditions

We've learned how to select data (columns and/or rows) based on their positions within a dataframe or the names of the columns/rows.  Now we will discuss how to select rows based on whether or not their values meet certain conditions.

For example, say we wanted to select all genes whose expression level in condition 1 was at least 20.

This can be done using the **.query** method.

Query accepts a string as an argument.  Pandas parses this string to evaluate whether or not rows meet the conditions within the string.  Standard boolean operators like **and** and **or** can be used within the string as **&** and **|**, respectively.

In [ ]:
#Defining the dataframe
d = {'Gene': ['GeneA', 'GeneB', 'GeneC', 'GeneD', 'GeneE', 'GeneF'],
    'cond1exp' : [100, 50, 40, 20, 1, 1000],
    'cond2exp' : [200, 12.5, 45, 20, 4, 800],
    'log2FC' : [1, -2, 0.17, 0.00, 2.00, -0.32],
    'pvalue' : [6e-5, 2e-6, 0.55, 1.00, 0.15, 0.001]}
    
df = pd.DataFrame(data = d)

#Select rows whose expression in condition 1 is 100
newdf = df.query('cond1exp == 100')

print(newdf.head())

In [ ]:
#Select rows whose expression in condition 1 is at least 20
newdf = df.query('cond1exp >= 20')

print(newdf.head())

In [ ]:
#Select rows whose expression in condition 1 is at least 20 and the pvalue is less than 0.05
newdf = df.query('cond1exp >= 20 & pvalue < 0.05')

print(newdf.head())

## Adding new columns

Suppose we wanted to add a new column of data.

This can be done with the **.assign** method.

    newdf = df.assign(colname = <list of column values>)

In [ ]:
#Defining the dataframe
d = {'Gene': ['GeneA', 'GeneB', 'GeneC', 'GeneD', 'GeneE', 'GeneF'],
    'cond1exp' : [100, 50, 40, 20, 1, 1000],
    'cond2exp' : [200, 12.5, 45, 20, 4, 800],
    'log2FC' : [1, -2, 0.17, 0.00, 2.00, -0.32],
    'pvalue' : [6e-5, 2e-6, 0.55, 1.00, 0.15, 0.001]}
    
df = pd.DataFrame(data = d)

#Add the expression in a third condition

cond3expvalues = [20, 50, 10, 1000, 10, 2] #these are IN ORDER

newdf = df.assign(cond3exp = cond3expvalues)
print(newdf.head())

Suppose we wanted to add a new column, but it's values were based on the values of other columns.  Let's say we wanted to add a new column called 'otherlog2FC'.  This is the same as the log2FC we currently have, but instead of log2(cond2exp / cond1exp), we want it to be log2(cond1exp/cond2exp).

In [ ]:
import numpy as np

#A new log2FC column

#First we need to get a list of the values that will make up our new column
cond1exp = df['cond1exp'] #this is a pandas series of the values in the cond1exp column, essentially a one-column dataframe
cond2exp = df['cond2exp']
newlog2FCvalues = np.log2(cond1exp / cond2exp)
print(newlog2FCvalues)

#Now add these values as a new column
newdf = df.assign(newlog2FC = newlog2FCvalues)
print(newdf.head())

## Adding new rows

Adding new rows is essentially the same as appending one data frame (even if it's only one row long) to the end of another data frame.  Not surprisingly, then, this is done with the **.append** method.


In [ ]:
#We now have data for GeneG

d = {'Gene': ['GeneG'], 'cond1exp': [10], 'cond2exp': [40], 'log2FC' : [2], 'pvalue' : [4e-5]}
df2 = pd.DataFrame(data = d)
print(df2.head(), '\n\n\n\n')

#Add df2 to the bottom of df
newdf = df.append(df2)
print(newdf, '\n\n\n\n')

#Oh no, df2 kept it's original index (0)! We want it to have a new index based on its position in the new df
newdf = df.append(df2, ignore_index = True)
print(newdf)

## Iterating through rows

Perhaps you want to iterate through rows, performing some kind of action on the data in each row.  To do this, we can use the **.iterrows()** method.  This returns a generator which yields both the index and the row.

In [ ]:
#Defining the dataframe
d = {'Gene': ['GeneA', 'GeneB', 'GeneC', 'GeneD', 'GeneE', 'GeneF'],
    'cond1exp' : [100, 50, 40, 20, 1, 1000],
    'cond2exp' : [200, 12.5, 45, 20, 4, 800],
    'log2FC' : [1, -2, 0.17, 0.00, 2.00, -0.32],
    'pvalue' : [6e-5, 2e-6, 0.55, 1.00, 0.15, 0.001]}
    
df = pd.DataFrame(data = d)
print(df)

#For each row, print the cond1exp value and cond2exp value
for index, row in df.iterrows():
    print(row['cond1exp'], row['cond2exp'])

#You can use this to add columns if you like
#Think of index as the rowname
for index, row in df.iterrows():
    totalexpression = row['cond1exp'] + row['cond2exp']
    #Add this as a new column named 'totexp' using .loc.
    if totalexpression < 50:
        df.loc[index, 'totexp'] = 'low'
    elif totalexpression >= 50 and totalexpression < 100:
        df.loc[index, 'totexp'] = 'medium'
    elif totalexpression > 100:
        df.loc[index, 'totexp'] = 'high'
    
print(df)

## Merging dataframes together

Often it is useful to combine datasets together.  For example, let's say someone else did a similar experiment.  You might want to take their data and compare it to yours to see how similar it is.  You can do this by merging your tables together.  This must be done on a key.  For example, say your data is below:

| Gene | expression in condition 1 | expression in condition 2 | log2 fold change | pvalue |
|------|---------------------------|---------------------------|------------------|--------|
|GeneA |       100                 | 200                       |        1         |   6e-5 |
|GeneB |    50                     | 12.5                      |        -2        |  2e-6  |
|GeneC |       40                  |   45                      |   0.17           | 0.55   |
|GeneD |       20                  |   20                      |   0.00           | 1.00   |
|GeneE |        1                  |   4                       |   2.00           | 0.15   |
|GeneF |       1000                |   800                     |   -0.32          | 0.001  |

and your friend's data looks like this:

| Gene | expression in condition 1 | expression in condition 2 | log2 fold change | pvalue |
|------|---------------------------|---------------------------|------------------|--------|
|GeneA |       120                 | 210                       |        0.81         |   2e-3 |
|GeneB |    50                     | 25                      |        -1        |  1e-4  |
|GeneC |       30                  |   25                      |   0.26           | 0.43   |
|GeneD |       20                  |   30                      |   -0.58           | 0.25   |
|GeneE |        2                  |   5                       |   1.32           | 0.31   |
|GeneF |       950                |   600                     |   -0.66          | 0.005  |

It might be useful to combine these into one table so that you can look at all the values for a particular gene at once.

This can be handled with the **.merge** function.  Merge requires a 'key' (or keys) to merge on.  These are the items that you want to use a reference points that join the two dataframes.  In this case, a key that makes sense is 'Gene'.  We want each row to be one gene followed by all the data associated with it.

    mergeddf = pd.merge(leftdf, rightdf, how = <string>, on = <string or list>)
    
The 'how' argument wants a string telling how the merge should be done.  The options are (left, right, outer, inner).  More on that in a bit.  The on argument is the name of the column you intend to use as the key for merging.  In this case, we would like it to be 'Gene'.  If you would like to join on two or more keys simultaneously, provide multiple strings in a list.

### 'How' to merge

As stated above, the four ways you can join your tables together are left, right, outer, and inner.  These have to do with dealing with keys that are present in one table but not the other.  Joining 'left' will return a merged dataframe that only has keys (rows) that were present in the left table.  Those that were present in the right table *but not in the left table* are not included here.  Joining 'right' works similarly but only includes keys that were present in the right table.  Joining 'inner' only includes keys that are present in both (the intersection).  Joining 'outer' includes keys that were present in either (the union).  
![alt text](https://shanelynnwebsite-mid9n9g1q9y8tt.netdna-ssl.com/wp-content/uploads/2017/03/join-types-merge-names.jpg "Title")

In this first example of merging our two gene expression tables together, how we join them does not matter since every key in one table is also present in the other.  However, you can imagine scenarios (essentially every time you would want to do this with real data) where this would not be the case.

In [ ]:
#Merge your two dataframes using the gene names as the key.

d = {'Gene': ['GeneA', 'GeneB', 'GeneC', 'GeneD', 'GeneE', 'GeneF'],
    'cond1exp' : [100, 50, 40, 20, 1, 1000],
    'cond2exp' : [200, 12.5, 45, 20, 4, 800],
    'log2FC' : [1, -2, 0.17, 0.00, 2.00, -0.32],
    'pvalue' : [6e-5, 2e-6, 0.55, 1.00, 0.15, 0.001]}

df = pd.DataFrame(data = d) #your data

d2 = {'Gene': ['GeneA', 'GeneB', 'GeneC', 'GeneD', 'GeneE', 'GeneF'],
    'cond1exp' : [120, 50, 30, 20, 2, 950],
    'cond2exp' : [210, 25, 25, 30, 5, 600],
    'log2FC' : [0.81, -1, 0.26, -0.58, 1.32, 0.66],
    'pvalue' : [2e-3, 1e-4, 0.43, 0.25, 0.31, 0.005]}

df2 = pd.DataFrame(data = d2) #your friend's data

mergeddf = pd.merge(df, df2, how = 'left', on = 'Gene')

print(mergeddf)

Looks good, but if possible, I'd like to know which value came from my experiment and which came from my friends.  Right now, these are indicated by the _x and _y suffixes, but we can give more meaningful suffixes.

In [ ]:
mergeddf = pd.merge(df, df2, how = 'left', on = 'Gene', suffixes = ['_mine', '_myfriends'])

print(mergeddf)

## Reshaping data

In general, tabular data can either be in 'wide' or 'long' format.  These are illustrated below:

This is a 'wide' table.  Each data variable has its own column.  It's perhaps more human readable, but addition of a new variable will require the creation of a new column.  

| Name | Age | Weight |
|------|---------------------------|---------------------------|
|Bob |       32                 | 128                       |
|Alice |    24                     | 86                      |
|Steve |       64                  |   95                     |


Most of the time, when you have multiple variables related to a single item, it can be easier to have the data in 'long' format.

| Name | Variable | Value |
|------|---------------------------|---------------------------|
|Bob |       Age                 | 32                       | 
|Bob |    Weight                 | 128                      |
|Alice |       Age                  |   24                      | 
|Alice |       Weight                 |   86                      | 
|Steve |        Age              |   64                       | 
|Steve |       Weight                |   95                     | 

This is the same data, just in a different form.  Addition of a new variable (say...gender) does not require the creation of a new column. Most plotting functions will prefer a table in long format over wide format.

The **melt()** function easily transforms wide data to long data.

    melteddf = pd.melt(df, id_vars, value_vars)
    
id_vars is a list of the column (or columns) you wish to use as the identifier.  In the example above, this would be ['Name'].  value_vars is a list of the column (or columns) you wish to retain values of.  In the example above, this would be ['Age', 'Weight'].  By default, this is all columns that are *not* in the id_vars list.



In [ ]:
#Melt our gene expression df, keeping the data associated with each variable

d = {'Gene': ['GeneA', 'GeneB', 'GeneC', 'GeneD', 'GeneE', 'GeneF'],
    'cond1exp' : [100, 50, 40, 20, 1, 1000],
    'cond2exp' : [200, 12.5, 45, 20, 4, 800],
    'log2FC' : [1, -2, 0.17, 0.00, 2.00, -0.32],
    'pvalue' : [6e-5, 2e-6, 0.55, 1.00, 0.15, 0.001]}

df = pd.DataFrame(data = d)
print(df, '\n\n\n\n')

melteddf = pd.melt(df, id_vars = ['Gene'], value_vars = ['cond1exp', 'cond2exp', 'log2FC', 'pvalue'])
print(melteddf)

## Plotting

OK so we've done our analysis, we've filtered and merged our dataframe so it's just like we want.  Now it's time for the payoff: plotting.  There are a few plotting packages written for Python, but we will be using **seaborn**.  It is still being actively developed, is reasonably mature, and is definitely the most beautiful.  You can read more about everything seaborn can do at https://seaborn.pydata.org/.

For now, we are just going to get our feet wet by making some simple plots.  The seaborn tutorial (https://seaborn.pydata.org/tutorial.html) is quite good, and to be honest, better than what we could cover ourselves here.  We will focus here on making plots of our toy data to get you initiated.

Seaborn plots can take pandas dataframes as inputs in the 'data' argument.  Let's try to make a barplot of our gene expression values.

> Note: this is not a particularly good analysis.  Don't do it in your next paper.  This is just for fun.

As a reminder, here's our data again:

| Gene | expression in condition 1 | expression in condition 2 | log2 fold change | pvalue |
|------|---------------------------|---------------------------|------------------|--------|
|GeneA |       100                 | 200                       |        1         |   6e-5 |
|GeneB |    50                     | 12.5                      |        -2        |  2e-6  |
|GeneC |       40                  |   45                      |   0.17           | 0.55   |
|GeneD |       20                  |   20                      |   0.00           | 1.00   |
|GeneE |        1                  |   4                       |   2.00           | 0.15   |
|GeneF |       1000                |   800                     |   -0.32          | 0.001  |

> Note 2: After we are done with the code to make the plot, we need one more line, plt.show(), to get the plot to actually show.  Many aesthetic details (axis labels, grids, etc.) can be added to the plot after making it, but before showing it.  Seaborn is based on an earlier plotting package called matplotlib.  Aesthetic changes to seaborn plots can therefore be made with matplotlib commands.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

#Our data
d = {'Gene': ['GeneA', 'GeneB', 'GeneC', 'GeneD', 'GeneE', 'GeneF'],
    'cond1exp' : [100, 50, 40, 20, 1, 1000],
    'cond2exp' : [200, 12.5, 45, 20, 4, 800],
    'log2FC' : [1, -2, 0.17, 0.00, 2.00, -0.32],
    'pvalue' : [6e-5, 2e-6, 0.55, 1.00, 0.15, 0.001]}

df = pd.DataFrame(data = d)

#We want a 'barplot'

myplot = sns.barplot(data = df, x = 'Gene', y = 'cond1exp')
plt.show()

#Nice, but I like red bars.
myplot = sns.barplot(data = df, x = 'Gene', y = 'cond1exp', color = 'red')
plt.show()


Not bad.  But let's say I want to see their expression in condition1 *and* condition2 in the same plot.  You'll notice that in the sns.barplot() call, there is only *x* and *y* arguments that determine what goes on each axis.  So, with the table as it is now, how can we show **both** conditions?  The short answer is you can't.  However, if we reorganized the table a bit, we could.  In fact, if we **melted** this table to it from wide into long data, we could.  See why below.

This is our original data:

| Gene | cond1exp | cond2exp | log2FC | pvalue |
|------|---------------------------|---------------------------|------------------|--------|
|GeneA |       100                 | 200                       |        1         |   6e-5 |
|GeneB |    50                     | 12.5                      |        -2        |  2e-6  |
|GeneC |       40                  |   45                      |   0.17           | 0.55   |
|GeneD |       20                  |   20                      |   0.00           | 1.00   |
|GeneE |        1                  |   4                       |   2.00           | 0.15   |
|GeneF |       1000                |   800                     |   -0.32          | 0.001  |

It's obvious that the x axis should be 'Gene', but what should the y axis be?  Condition1 or condition2? Remember that we'd like to show both on the same plot.  If we melted the table to make it long we would come up with this:

| Gene | variable | value |
|------|---------------------------|---------------------------|
|GeneA |       cond1exp                 | 100                       |
|GeneB |    cond1exp                     | 50                      |
|GeneC |       cond1exp                  |   40                      |
|GeneD |       cond1exp                  |   20                      |
|GeneE |        cond1exp                  |   1                       |
|GeneF |       cond1exp                |   1000                     |
|GeneA |       cond2exp                |   200                     |
|GeneB |       cond2exp                |   12.5                     |
|GeneC |       cond2exp                |   45                     |
|GeneD |       cond2exp                |   20                     |
|GeneE |       cond2exp                |   4                     |
|GeneF |       cond2exp                |   800                     |

Now, the x axis can still be 'Gene' and the y axis can be 'value'.  All we need to do then is come up with a way of differentiating based on the value in 'variable'.


In [ ]:
d = {'Gene': ['GeneA', 'GeneB', 'GeneC', 'GeneD', 'GeneE', 'GeneF'],
    'cond1exp' : [100, 50, 40, 20, 1, 1000],
    'cond2exp' : [200, 12.5, 45, 20, 4, 800],
    'log2FC' : [1, -2, 0.17, 0.00, 2.00, -0.32],
    'pvalue' : [6e-5, 2e-6, 0.55, 1.00, 0.15, 0.001]}

df = pd.DataFrame(data = d)

#Melt df

melteddf = pd.melt(df, id_vars = ['Gene'], value_vars = ['cond1exp', 'cond2exp'])
print(melteddf)

myplot = sns.barplot(data = melteddf, x = 'Gene', y = 'value', hue = 'variable')
plt.show()

Not bad.  Maybe instead we want to make a boxplot of all the genes within a sample.  

In [ ]:
d = {'Gene': ['GeneA', 'GeneB', 'GeneC', 'GeneD', 'GeneE', 'GeneF'],
    'cond1exp' : [100, 50, 40, 20, 1, 1000],
    'cond2exp' : [200, 12.5, 45, 20, 4, 800],
    'log2FC' : [1, -2, 0.17, 0.00, 2.00, -0.32],
    'pvalue' : [6e-5, 2e-6, 0.55, 1.00, 0.15, 0.001]}

df = pd.DataFrame(data = d)

#Melt df

melteddf = pd.melt(df, id_vars = ['Gene'], value_vars = ['cond1exp', 'cond2exp'])

sns.boxplot(data = melteddf, x = 'variable', y = 'value')
plt.show()

Alright, now what about a scatter for comparing values from the two conditions?

In [ ]:

d = {'Gene': ['GeneA', 'GeneB', 'GeneC', 'GeneD', 'GeneE', 'GeneF'],
    'cond1exp' : [100, 50, 40, 20, 1, 1000],
    'cond2exp' : [200, 12.5, 45, 20, 4, 800],
    'log2FC' : [1, -2, 0.17, 0.00, 2.00, -0.32],
    'pvalue' : [6e-5, 2e-6, 0.55, 1.00, 0.15, 0.001]}

df = pd.DataFrame(data = d)

#No need to melt this time.  x = cond1, y = cond2

#plot their relation with a relplot

sns.relplot(data = df, x = 'cond1exp', y = 'cond2exp')
plt.show()

## A message about overplotting

Scatterplots are common in genomic analysis.  Often, you are dealing with tens or hundreds of thousands of observations, and scatterplots are quick and easy ways to look for relationships between variables.  However, they should be used with some caution.  One common pitfall of this type of plot is overplotting.  This occurs when the density of points on a plot obscures the true nature of the data.

Consider the scatterplot below.  It seems obvious that there is a correlation between x and y (there is).  It also looks like the density of the data in the x direction is more or less constant between x = 0 and x = 6.

![alt text](scatter.png)

However, this is misleading.  It would be nice to somehow represent the density of the dots at each location.  This would give us information about how much of the data lies at each point.  One way this can be done is by using a hexbin plot.  We aren't going to get into how to make them right now, but you can read their documentation [here](https://seaborn.pydata.org/examples/hexbin_marginals.html).

Using this approach, we can see that, in actuality, the vast majority of the data is contained between x = 0 and x = 2.  

![hexbin](hexbin.png)

# Exercises

## Exercise 1

Often, it's not a good idea to work on raw expression values.  Work on tranformed (for example, log-transformed) can sometimes be more informative (if not more mathematically correct).  The scatter plot we produced to compare gene expression values in conditions 1 and 2 is a good example.  It would be better to instead look at the correlation between logged (let's say log10) values.  Produce a scatter plot comparing the logged expression values.

Hints:
- Add two new columns to the dataframe that contain the logged values in conditions 1 and 2, respectively.
- You may (will) need the numpy.log10() function.

In [ ]:
d = {'Gene': ['GeneA', 'GeneB', 'GeneC', 'GeneD', 'GeneE', 'GeneF'],
    'cond1exp' : [100, 50, 40, 20, 1, 1000],
    'cond2exp' : [200, 12.5, 45, 20, 4, 800],
    'log2FC' : [1, -2, 0.17, 0.00, 2.00, -0.32],
    'pvalue' : [6e-5, 2e-6, 0.55, 1.00, 0.15, 0.001]}

df = pd.DataFrame(data = d)

## Exercise 2

Your friend has done a very similar experiment, looking at the expression of the same genes in the same conditions.  You might like to know how well your two datasets agree with each other.  One way to do this would be to correlate the log2 fold changes that you observed with the log2 fold changes that your friend observed.  Write code to obtain the Spearman correlation coefficient for your two log2 fold changes.  

Now, is the correlation stronger or weaker than what you would expect by chance?  One way to find this out would be to randomize your data and get correlations for the random data.  If you did this many times (say, 500 times) you could see where your real correlation coefficient fell within the distribution of control random correlations.  Write code to do this, finishing with a z-score comparison of the real correlation coefficient and the random values.

> Note: A z-score compares a single value (like your real corrleation) to a distribution of other values (like all the random correlations).  Lets define your single value as x and the distribution of the other values as y.  The z-score is defined as (mean(y) - x) / (standarddeviation(y))

Hints:
- You can get the spearman correlation between two lists of numbers using scipy.stats.spearmanr().  This function assumes that the order of the values in each list is paired. Think of it like a scatter plot.  For each point, the value in the first list is the x coordinate and the value in the second list is its y coordinate.
- You can randomly shuffle the order of values within a list using shuffle(), which is in the random package.  Importantly, this shuffles the list **in place**.
- You can get the mean of a distribution (also known as a list) with mean(), which is in the numpy package.
- You can get the standard deviation of a distribution (also known as a list) with std(), which is in the numpy package.


In [ ]:
from numpy import mean
from numpy import std
from random import shuffle
from scipy.stats import spearmanr

#Your data
d = {'Gene': ['GeneA', 'GeneB', 'GeneC', 'GeneD', 'GeneE', 'GeneF'],
    'cond1exp' : [100, 50, 40, 20, 1, 1000],
    'cond2exp' : [200, 12.5, 45, 20, 4, 800],
    'log2FC' : [1, -2, 0.17, 0.00, 2.00, -0.32],
    'pvalue' : [6e-5, 2e-6, 0.55, 1.00, 0.15, 0.001]}

yourdata = pd.DataFrame(data = d)

#Your friend's data
d2 = {'Gene': ['GeneA', 'GeneB', 'GeneC', 'GeneD', 'GeneE', 'GeneF'],
    'cond1exp' : [120, 50, 30, 20, 2, 950],
    'cond2exp' : [210, 25, 25, 30, 5, 600],
    'log2FC' : [0.81, -1, 0.26, -0.58, 1.32, 0.66],
    'pvalue' : [2e-3, 1e-4, 0.43, 0.25, 0.31, 0.005]}

friendsdata = pd.DataFrame(data = d2)


mylog2 = yourdata.loc[:, 'log2FC'].values
friendlog2 = friendsdata.loc[:, 'log2FC'].values

realcorr = spearmanr(mylog2, friendlog2)[0]

randomcors = []
for i in range(500):
    shuffle(mylog2) #shuffles in place
    shuffle(friendlog2) #shuffles in place
    randomcor = spearmanr(mylog2, friendlog2)[0]
    randomcors.append(randomcor)
    
randommean = mean(randomcors)
randomsd = std(randomcors)

z = (realcorr - randommean) / randomsd
print(z)